##### Created on 06/16/2022
##### plot bias vs resolution in cmip6
##### using merra2 TS and gcm ts
##### my code to read GCMs 

In [23]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.cm import get_cmap
from myReadGCMsDaily import read_var_mod
import calendar
from global_land_mask import globe
import glob
import math
from scipy.interpolate import griddata
from scipy import stats

In [2]:
#####Constants
Cp = 1004           #J/kg/K
Rd = 287            #J/kg/K
con= Rd/Cp

In [3]:
latr1 = 30
latr2 = 70

In [4]:
modname = ['CESM2','CESM2-WACCM','HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','CNRM-ESM2-1','INM-CM5-0','MPI-ESM1-2-HR','UKESM1-0-LL','MPI-ESM1-2-LR','MPI-ESM-1-2-HAM','CMCC-CM2-SR5','CMCC-CM2-HR4','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','IPSL-CM5A2-INCA']
varname = ['sfcWind', 'ts','psl'] #'sfcWind', 'hfss', 'hfls', 'tas', 'ps', 'psl',,'pr'
pvarname= ['ta']


In [16]:
l=0
m=1

In [17]:
time1=[2010, 1, 1]
time2=[2012, 12, 30]

In [18]:
# try:
for j in range(l,m): #
    print(modname[j])
    for i in varname:
        locals()[i+'__'+str(j+1)] = read_var_mod('surface', modname[j], 'historical', i, time1, time2)
        # print(i)
    for k in pvarname:
        locals()[k+'__'+str(j+1)] = read_var_mod('p_level', modname[j], 'historical', k, time1, time2)
        
# except IndexError:
#     print(str(j+1)+' not available')
print('done')    

CESM2


/glade/work/geethma/research/codes/myReadGCMsDaily.py:60: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  data.extend(np.array(datai[ind1:ind2+1,:,:]))


done


In [19]:
# i = l
for i in range(l,m):
    lat  = locals()['sfcWind__'+str(i+1)][0]
    lon  = locals()['sfcWind__'+str(i+1)][1]
    time = locals()['sfcWind__'+str(i+1)][2]
    
    g_lat_diff = np.abs(lat[1]-lat[0])
    g_lon_diff = np.abs(lon[1]-lon[0])
    locals()['g_res'+str(i+1)] = (np.sqrt(g_lat_diff**2 + g_lon_diff**2)) * 110.574

    for j in varname:
        locals()[j+str(i+1)] = locals()[j+'__'+str(i+1)][4]
        locals()[j+str(i+1)] = np.ma.filled(locals()[j+str(i+1)], fill_value=np.nan)
        print(np.shape(locals()[j+str(i+1)]))
# 
    for k in pvarname:
        locals()[k+str(i+1)] = locals()[k+'__'+str(i+1)][4]
        locals()[k+str(i+1)] = np.ma.filled(locals()[k+str(i+1)], fill_value=np.nan)
        print(np.shape(locals()[k+str(i+1)]))
        
        locals()['lev'+str(i+1)] = locals()['ta__'+str(i+1)][3]

(1094, 192, 288)
(1094, 192, 288)
(1094, 192, 288)
(1094, 8, 192, 288)


In [20]:
print(lev1[1], lev1[2])

85000.0 70000.0


In [21]:
for i in range(l,m):
    x_lat = np.array(lat)
    lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
    lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]    
    lats = lat[lat_ind1[0]:lat_ind2[0]]

    x_lon = lon
    lons = np.array(lon)
    lons[lons > 180] = lons[lons > 180]-360
    
    maskm = np.ones((len(time),len(lats),len(lon)))

    for a in range(len(lats)):
        for b in range(len(lons)):
            if globe.is_land(lats[a], lons[b])==True:
                maskm[:,a,b] = math.nan
 
 ###  averaged theta at 800hPa and surface 
    theta_850 = locals()['ta'+str(i+1)][:,1,:,:]*(100000/85000)**con
    theta_700 = locals()['ta'+str(i+1)][:,2,:,:]*(100000/70000)**con
    theta_800 = theta_700 + ((2/3) * (theta_850 - theta_700))
    
    theta_sfc = locals()['ts'+str(i+1)]*(100000/locals()['psl'+str(i+1)])**con

### CAOI at 800hPa
    M = theta_sfc - theta_800
    
    x_sfcWind = locals()['sfcWind'+str(i+1)]
    m_sfcWind = x_sfcWind[:,lat_ind1[0]:lat_ind2[0],:]
    lats = lat[lat_ind1[0]:lat_ind2[0]]

    x_M = M
    m_M = x_M[:,lat_ind1[0]:lat_ind2[0],:]
    
    cao = np.array(m_M)
    sw  = np.array(m_sfcWind)
    
    plot_CAOI = np.array(np.multiply(maskm,cao))
    wind      = np.array(np.multiply(maskm,sw))
    
    # pl_theta  = plot_CAOI #.reshape(-1)
    # pl_wind   = wind #.reshape(-1)
    
    # plo_theta = pl_theta[pl_theta>-40]
    # plo_wind  = pl_wind[pl_theta>-40]
    
#     plot_theta = plo_theta[plo_theta<40]
#     plot_wind  = plo_wind[plo_theta<40]
    
#     ind = np.argsort(plot_theta)
#     xx = np.sort(plot_theta)
#     yy = plot_wind[ind]
    
#     g_indx = np.isnan(xx*yy)==False

#     g_M = xx[g_indx]
#     g_W = yy[g_indx]

#     g_re_theta  = g_M[g_M>2.5]
#     g_re_wind   = g_W[g_M>2.5]
#     g_res_theta = g_re_theta[g_re_theta<3.5]
#     g_res_wind  = g_re_wind[g_re_theta<3.5]
    
    # g_bin_means, g_bin_edges, g_binnumber = stats.binned_statistic_2d(plot_CAOI, plot_CAOI, 'mean', bins=500)
    

In [27]:
o_M = np.nanmean(obs_plot_CAOI)

In [26]:
idx = np.isnan(plot_CAOI[:,:,:] * obs_plot_CAOI[:,:,:])==False

ValueError: operands could not be broadcast together with shapes (1094,43,288) (1096,40,360) 

In [25]:
# g_bin_means, g_bin_edges, g_binnumber = stats.binned_statistic_2d(plot_CAOI, plot_CAOI, 'mean', bins=500)

In [28]:
o_M

-10695284372287.291

In [5]:
import glob
merlist = np.sort(glob.glob('../data_merra/all_lat_lon/levels/MERRA2_*.nc'))
sfclist = np.sort(glob.glob('../data_merra/all_lat_lon/surface_old/MERRA2_*.nc'))
maclist = np.sort(glob.glob('../MACLWP_dailymean/take/wind1deg*.nc4'))

In [6]:
import netCDF4 as nc
import xarray as xr
p_mer_T   = []
p_mac_w   = []
sfc_mer_T = []
sfc_mer_P = []

for i in range(len(merlist)): #len(merlist)
    d_path = merlist[i]
    data   = nc.Dataset(d_path)
    # print(d_path)
    
    if i==0:
        merlat = data.variables['lat'][:]
        merlon = data.variables['lon'][:]
        merlev = data.variables['lev'][:]
        #shape latitude
        mer_lat = np.flip(merlat)
        mer_lat = np.array(mer_lat)
        mlat_ind1 = np.where(mer_lat == mer_lat.flat[np.abs(mer_lat - (latr1)).argmin()])[0]
        mlat_ind2 = np.where(mer_lat == mer_lat.flat[np.abs(mer_lat - (latr2)).argmin()])[0]
        p_mer_lat  = np.array(mer_lat[mlat_ind1[0]:mlat_ind2[0]])
        #shape longitude
        merlon[merlon > 180] = merlon[merlon > 180]-360
        # mer_lon = np.array(merlon)
        
    merT   = data.variables['T'][:] #(time, lev, lat, lon)
    mer_T = np.array(merT[:,:,::-1,:])
    p_mer_T.extend(mer_T[:,:,mlat_ind1[0]:mlat_ind2[0],:])
    
temp = np.array(p_mer_T)

for i in range(len(sfclist)): #len(merlist)
    s_path = sfclist[i]
    sdata  = nc.Dataset(s_path)
    # print(d_path)
    
    if i==0:
        sfclat = sdata.variables['lat'][:]
        sfclon = sdata.variables['lon'][:]
        #shape latitude
        sfc_lat = np.flip(sfclat)
        sfc_lat = np.array(sfc_lat)
        flat_ind1 = np.where(sfc_lat == sfc_lat.flat[np.abs(sfc_lat - (latr1)).argmin()])[0]
        flat_ind2 = np.where(sfc_lat == sfc_lat.flat[np.abs(sfc_lat - (latr2)).argmin()])[0]
        p_sfc_lat  = np.array(sfc_lat[flat_ind1[0]:flat_ind2[0]])
        #shape longitude
        sfclon[sfclon > 180] = sfclon[sfclon > 180]-360
        # sfc_lon = np.array(sfclon)
        
    sfcT   = sdata.variables['TS'][:] 
    sfc_T = np.array(sfcT[:,::-1,:])
    sfc_mer_T.extend(sfc_T[:,flat_ind1[0]:flat_ind2[0],:])
    
    sfcP   = sdata.variables['SLP'][:] 
    sfc_P = np.array(sfcP[:,::-1,:])
    sfc_mer_P.extend(sfc_P[:,flat_ind1[0]:flat_ind2[0],:])
    
sfctemp = np.array(sfc_mer_T)
sfcpres = np.array(sfc_mer_P)

for i in range(len(maclist)): #len(maclist)
    ddpath = maclist[i]
    ddata  = nc.Dataset(ddpath)
    macw   = ddata.variables['sfcwind'][:] #(time,lat,lon)
    # print(ddpath)
    
    if i==0:
        maclat = ddata.variables['lat'][:]
        maclon = ddata.variables['lon'][:]
        #shape latitude
        mac_lat = np.array(maclat)
        slat_ind1 = np.where(mac_lat == mac_lat.flat[np.abs(mac_lat - (latr1)).argmin()])[0]
        slat_ind2 = np.where(mac_lat == mac_lat.flat[np.abs(mac_lat - (latr2)).argmin()])[0]
        p_mac_lat  = np.array(mac_lat[slat_ind1[0]:slat_ind2[0]])
        #shape longitude
        maclon[maclon > 180] = maclon[maclon > 180]-360
        # mac_lon = np.array(maclon)

    n_w = macw[:,slat_ind1[0]:slat_ind2[0],:]
    p_mac_w.extend(n_w)

wind = np.array(p_mac_w)


In [7]:
p_level = 1
theta_800 = np.array(np.multiply(temp[:,p_level,:,:], (100000/(merlev[p_level]*100))**(Rd/Cp)))
theta_sfc = np.array(np.multiply(sfctemp, (100000/sfcpres)**(Rd/Cp)))
p_CAOI = np.array(np.subtract(theta_sfc,theta_800))


In [8]:
maskm = np.ones((len(temp),len(p_mer_lat),len(merlon)))

for a in range(len(p_mer_lat)):
    for b in range(len(merlon)):
        if globe.is_land(p_mer_lat[a], merlon[b])==True:
            maskm[:,a,b] = math.nan

In [9]:
caoi_test = p_CAOI
wind_test = wind

In [11]:
obs_plot_CAOI = np.ones((len(temp),len(p_mer_lat),len(merlon)))
obs_plot_CAOI[:,:,180:360] = caoi_test[:,:,0:180]
obs_plot_CAOI[:,:,0:180]   = caoi_test[:,:,180:360]
obs_plot_CAOI = np.array(obs_plot_CAOI)

In [12]:
obs_plot_wind = np.ones((len(temp),len(p_mer_lat),len(merlon)))
obs_plot_wind[:,:,180:360] = wind_test[:,:,0:180]
obs_plot_wind[:,:,0:180]   = wind_test[:,:,180:360]
obs_plot_wind = np.array(obs_plot_wind)

In [13]:
obs_plot_CAOI = np.array(np.multiply(maskm,obs_plot_CAOI))
obs_plot_wind = np.array(np.multiply(maskm,obs_plot_wind))

In [14]:
np.shape(obs_plot_CAOI)

(1096, 40, 360)

In [15]:
np.shape(obs_plot_wind)

(1096, 40, 360)

In [ ]:
plot_indx = np.isnan(plot_CAOI*plot_CAOI)==False

In [ ]:
plot_mer_theta = plot_CAOI[plot_indx]
plot_mac_wind  = plot_wind[plot_indx]

In [ ]:
ind = np.argsort(plot_mer_theta)
xx = np.sort(plot_mer_theta)
yy = plot_mac_wind[ind]

In [88]:
from scipy import stats
x = np.array(lon)
y = np.array(lats)
z = np.array(time)
zz, yy, xx = np.meshgrid(z,y,x,indexing='xy')
binx = np.arange(0,360,1)
biny = np.arange(30,70,1)
binz = np.arange(0,len(time),1)

data = plot_CAOI
# idx = np.isnan(data[j,:,:])==False
interp_data = stats.binned_statistic_dd(zz,xx, yy, data, statistic='mean', bins=[binz,binx,biny])

# for j in range(len(time)):
    # data = plot_CAOI
    # idx = np.isnan(data[j,:,:])==False
    # interp_data = stats.binned_statistic_dd(xx[idx].ravel(), yy[idx].ravel(), data[idx].ravel(), statistic='mean', bins=[binx,biny]).statistic

TypeError: binned_statistic_dd() got multiple values for argument 'statistic'

In [80]:
from scipy.interpolate import interpn
from scipy.interpolate import RegularGridInterpolator
xa = np.arange(0,360,1)
ya = np.arange(30,70,1)
# za = np.array(time)
yga, xga = np.meshgrid(ya, xa, indexing='ij', sparse=True)
interp_CAOI = []
for k in range(len(time)): #
    # interp = interpn((ya,xa), plot_CAOI[k,:,:], (yga,xga))
    interp = RegularGridInterpolator((ya,xa), plot_CAOI[k,:,:], method='nearest')
    interp_CAOI.append(interp)

ValueError: There are 40 points and 43 values in dimension 0

In [62]:
np.shape(plot_CAOI)

(1094, 43, 288)

In [69]:
np.shape(interp_CAOI)

(1094,)

In [76]:
lats[-1]

69.26701570680629

In [15]:
print(np.max(g_M), np.min(g_M))
print(np.max(g_W), np.min(g_W))

19.5496550061186 -39.989846186403724
29.761993408203125 0.01262784656137228


In [19]:
print(np.max(g_res_theta1), np.min(g_res_theta1))
# print(np.max(o_res_theta), np.min(o_res_theta))

3.499998823493911 2.5000117108659197
